In [22]:
import os

In [23]:
!ls

01_extract_text_from_xml.ipynb	datasets	    requirements.txt  yolo_pcb_dataset
copies_log.txt			kaggle_pcb_dataset  runs	      yolo_trainnig.ipynb
dataset_creator.py		readme.md	    ultralytics       yolov8n.pt


In [24]:
# !git clone https://github.com/ultralytics/ultralytics -b main

In [25]:
!pip install -r requirements.txt

# yolo tranning

In [29]:
os.chdir('/content/drive/Othercomputers/My Computer/Documentos_HD/Codes/yolo_free-code-camp_course/7_PCB_ERR_KAGGLE')
!yolo mode=train data='/content/drive/Othercomputers/My Computer/Documentos_HD/Codes/yolo_free-code-camp_course/7_PCB_ERR_KAGGLE/ultralytics/data.yaml' model=yolov8l.pt batch=32 name=pcb_err_model epochs=40 lr0=0.01

Ultralytics YOLOv8.2.20 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/Othercomputers/My Computer/Documentos_HD/Codes/yolo_free-code-camp_course/7_PCB_ERR_KAGGLE/ultralytics/data.yaml, epochs=40, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pcb_err_model8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False